In [1]:
import os
import sys

from pathbook.pathbook import *
labels = ['klikun', 'maliy', 'shipun']

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

### annotating initial dataset for detection and segmentation

In [12]:
columns=['set','path','label','x_min','y_min','x_max','y_max']
train_array = []
val_array = []

np.random.seed(17)
val_p = 0.15

for label_idx, label in enumerate(labels):
    mask_dir = os.path.join(path_initial_train_dataset, label, 'masks')
    image_dir = os.path.join(path_initial_train_dataset, label, 'images')

    for file_idx, mask in enumerate(tqdm(os.listdir(mask_dir))):
        impath = os.path.join(image_dir, mask.replace('.png','.jpg'))
        if not os.path.exists(impath):
            print(impath)
            continue
        image = cv2.imread(impath)
        mask = cv2.imread(os.path.join(mask_dir, mask))

        h, w, _ = image.shape
        h_, w_, _ = mask.shape
        if not (h == h_ and w == w_):
            continue

        save_dir=''
        val = (np.random.random() < val_p)
        if val:
            save_dir = path_val_dataset
        else:
            save_dir = path_train_dataset

        # "[scource: initial|extra|augmented]-[label: klikun|maliy|shipun]-[ingroup index]]"
        name = f'initial-{label}-{file_idx}'
        path = os.path.join(save_dir, name)
        cv2.imwrite(path+'.jpg', image)
        labelfile = open(path+'.txt','w')

        imgray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        colors = np.unique(imgray)[1:]
        for color in colors:
            _, thresh = cv2.threshold(imgray, color, color, type=cv2.THRESH_TOZERO_INV)
            _, thresh = cv2.threshold(thresh, color-1, color, type=cv2.THRESH_TOZERO)
            contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            seg = []
            for contour in contours:
                seg += (contour[:,0]/[w,h]).flatten().tolist()
            x = seg[0::2]
            y = seg[1::2]
            print(label_idx,*seg,file=labelfile)

            if val:
                val_array.append(['val',path+'.jpg',label,min(x),min(y),max(x),max(y)])
            else:
                train_array.append(['train',path+'.jpg',label,min(x),min(y),max(x),max(y)])

        labelfile.close()

pd.DataFrame(train_array, columns=columns).to_csv(path_train_annotation,index=False)
pd.DataFrame(val_array, columns=columns).to_csv(path_val_annotation,index=False)

 71%|███████   | 2152/3025 [01:19<00:24, 36.23it/s]

/Users/samedi/Desktop/Минприроды/klikun/images/original (60).jpg


 49%|████▉     | 1476/3002 [00:34<00:26, 57.16it/s]

/Users/samedi/Desktop/Минприроды/maliy/images/1960538708.jpg


100%|██████████| 3011/3011 [02:46<00:00, 18.08it/s]
